<a href="https://colab.research.google.com/github/agprasanna/Coursera_Capstone/blob/master/Opening_an_Indian_Restaurant_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the necesary packages (if not done already)


In [1]:
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.10.0 
#!conda install -c conda-forge geopandas
#!pip install geopandas

Import the necesary libraries here


In [2]:
import pandas as pd
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import numpy as np

import urllib.request
import requests

from geopy.geocoders import Nominatim    # import geocoder Nominatim
import folium                            # import map rendering 

import geopandas as gpd             #to read in shape file and provides high #level interface with #matplotlib library for making maps

import matplotlib.cm as cm          # Matplotlib and associated plotting modules
import matplotlib.colors as colors 

#import k-means from clustering stage
from sklearn.cluster import KMeans

print("libraries imported!")

libraries imported!


Importing the geojson file from the OpenData project of City of  Toronto

In [16]:
Toronto_geojson_url= "https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/a083c865-6d60-4d1d-b6c6-b0c8a85f9c15?format=geojson&projection=4326"

Toronto_Geojson = gpd.read_file(Toronto_geojson_url)
Toronto_Geojson.head(2)

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,6581,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
1,6582,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"POLYGON ((-79.41096 43.70408, -79.40962 43.704..."


Extracting only the relevant information from the above dataset

In [4]:
Toronto_Geojson1 = Toronto_Geojson[['AREA_SHORT_CODE','geometry']]
#Toronto_Geojson1.head()

In [40]:
# Determining the centre for the map

# Assigning centroids to map using the center point of the geojson data
#latitude= Toronto_Geojson.centroid.x.mean()
#longitude=Toronto_Geojson.centroid.y.mean()

address = 'Casa Loma, Ontario, Canada'

geolocator = Nominatim(user_agent="Toronto_map")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

In [33]:
Toronto_df = Toronto_Geojson[['AREA_SHORT_CODE','AREA_NAME','LONGITUDE','LATITUDE']]
#rename columns for convenience 
Toronto_df = Toronto_df.rename(columns={'AREA_SHORT_CODE': 'Code', 'AREA_NAME': 'Neighborhood', 'LONGITUDE': 'Longitude', 'LATITUDE': 'Latitude'})
Toronto_df['Neighborhood'] = Toronto_df['Neighborhood'].str.partition('(')
Toronto_df.head()

,Code,Neighborhood,Longitude,Latitude
0,94,Wychwood,-79.425515,43.676919
1,100,Yonge-Eglinton,-79.403590,43.704689
2,97,Yonge-St.Clair,-79.397871,43.687859
3,27,York University Heights,-79.488883,43.765736
4,31,Yorkdale-Glen Park,-79.457108,43.714672


In [34]:
Toronto_df.dtypes


Code              int64
Neighborhood     object
Longitude       float64
Latitude        float64
dtype: object

In [42]:
# create map of Toronto using latitude and longitude values created above
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10,width='80%',height='80%', min_zoom=10,max_bounds= True, min_lat=43.5,max_lat=44,min_lon=-79.8,max_lon=-78.9)

# add markers to map
#for lat, lng, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Neighborhood']):
#    label = '{}'.format(neighborhood)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='blue',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.6,
#        parse_html=False).add_to(map_Toronto)

# Creating choropleth map
folium.Choropleth(geo_data=Toronto_Geojson,
                       name='Choropleth',
                       data=Toronto_df,
                       columns=['Code','Longitude'],
                       key_on='feature.properties.area_short_code',
                       fill_color='YlGn',
                       fill_opacity=0.6,
                       line_opacity=0.3,
                       legend_name='Over all average taxable income of suburb',
                       smooth_factor=0,
                       highlight=True
                      ).add_to(map_Toronto)


folium.LayerControl().add_to(map_Toronto)

#folium.GeoJson(Toronto_Geojson)

map_Toronto